In [1]:
import scanpy as sc 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

sys.path.append('../../src')

In [2]:
adata = sc.read_h5ad('/ix/djishnu/shared/djishnu_kor11/training_data_2025/mouse_kidney_visium.h5ad')
adata

AnnData object with n_obs × n_vars = 415538 × 3058
    obs: 'ct3', 'cell_type', 'cell_type_int'
    obsm: 'spatial', 'spatial_unscaled'
    layers: 'imputed_count'

In [3]:
from spaceoracle.tools.network import RegulatoryFactory


In [4]:
# use the same links as the XYZeq2 kidney model

base_dir = '/ix/djishnu/shared/djishnu_kor11/'

co_grn = RegulatoryFactory(
    colinks_path=base_dir + 'training_data_2025/mouse_kidney_13_colinks.pkl',
    annot='cell_type_int'
)

In [5]:
sp_maps = np.load('/ix/djishnu/shared/djishnu_kor11/covet_outputs/mouse_kidney_visium/COVET_SQRT.npy')
feature_key = 'COVET_SQRT'
adata.obsm['COVET_SQRT'] = sp_maps

In [6]:
cov_genes = pd.read_csv('/ix/djishnu/shared/djishnu_kor11/covet_outputs/mouse_kidney_visium/CovGenes.csv', index_col=0)
cov_genes.values.flatten()

array(['Col3a1', 'Spp2', 'Ren1', 'Atp6v1g3', 'Rgs5', 'Snhg11', 'Slc13a3',
       'Fabp4', 'S100a6', 'S100a9', 'Egf', 'Sgms2', 'Calb1', 'Aqp3',
       'Tpm2', 'Clcnkb', 'Jchain', 'Cxcl9', 'Akr1b7', 'Tmem213', 'Igkc',
       'Emx1', 'Tmem52b', 'Mgp', 'Ptpro', 'Klk1', 'Rhcg', 'Nupr1',
       'Sfrp1', 'Aadat', 'Apela', 'Mt2', 'Slc12a3', 'Kcnj1', 'Tagln',
       'Plet1', 'Car12', 'Dcn', 'Pgam2', 'Igfbp3', 'Abca13', 'Col1a1',
       'Slc26a4', 'Igha', 'Klhl3', 'Sema3g', 'Clu', 'Ly6d', 'Pvalb',
       'Myh11', 'Car15', 'Kng2', 'Muc20', 'H2-eb1', 'C3', 'Slc8a1',
       'Aqp4', 'Slc4a9', 'Cd74', 'Slc14a2', 'Slc22a8', 'Acta2', 'Scd1',
       'S100g'], dtype=object)

In [7]:
cell_threshes = pd.read_parquet(
    base_dir + 'training_data_2025/kidney_visiumHD_LRs.parquet')
adata.uns['cell_thresholds'] = cell_threshes

In [ ]:
adata.var_names

In [ ]:
# see how well it estimates expression for well-expressed gene
from spaceoracle.models.adapted_estimators import PrefeaturizedCellularProgramsEstimator

target_gene = 'Sox17'
n_epochs = 100

estimator = PrefeaturizedCellularProgramsEstimator(
    adata, target_gene=target_gene, cluster_annot='cell_type_int',
    radius=100, contact_distance=30, grn=co_grn, 
    sp_maps_key='COVET_SQRT')

estimator.fit(
    num_epochs=n_epochs, 
    learning_rate=5e-3, 
    score_threshold=0.2,
    estimator='lasso',
    # use_anchors=False
)

In [ ]:
estimator.modulators

In [ ]:
# plot betadatas spatially
betadatas = estimator.betadatas

In [ ]:
from spaceoracle.astronomer import Astronaut

neil = Astronaut(
    adata=adata,
    annot='cell_type_int', 
    max_epochs=100, 
    learning_rate=5e-4, 
    spatial_dim=64,
    batch_size=512,
    grn=co_grn,
    radius=100,
    contact_distance=30,
    save_dir=base_dir + 'covet_runs/mouse_kidney_13'
)

neil.run()